# HW 1 - Building a network from scratch

* x1, x2, x3 = (1, 2, -1)
* Weights have been initated with 1. value
* Biases have been initated with 0 value
* The reuqired network must include 2 hidden layers, with 2 neurons in each layer
* One dimension output layer
* true y value = 1


## Loading required packages 

In [1]:
import pandas as pd
import numpy as np
import numpy.typing as npt

### Abstract Class - Layer

In [2]:
from abc import ABC, abstractclassmethod

class Layer(ABC):
    def __init__(self):
        self.input = None
        self.output = None
    
    @abstractclassmethod
    def forward_prop(self, input):
        pass
    
    @abstractclassmethod
    def backward_prop(self, input):
        pass

### Recall - 

    * DE/DX = (DE/DY)*W.T
    * DE/DW = X.T*(DE/DY)
    * DE/DB = DE/DY

### Fully Connected && Activation Layer Classes

In [175]:
class FullyConnectedLayer(Layer):
    def __init__(self, input_size, output_size, hw_1_init: bool = False):
        if hw_1_init:
            self.weights = np.ones((input_size, output_size)) 
            self.bias = np.zeros((1, output_size))
        else:
            self.weights = np.random.rand(input_size, output_size) - 0.5
            self.bias = np.random.rand(1, output_size) - 0.5
        
        
    def forward_prop(self, input_data: npt.ArrayLike) -> npt.ArrayLike:
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output
        
    def backward_prop(self, output_err: npt.ArrayLike, lr: float) -> npt.ArrayLike:
        input_err = np.dot(output_err, self.weights.T)
        weights_err = np.dot(self.input.T, output_err)
        print(f"{'*'*13}\nDLoss/DW:\n{weights_err}\n{'*'*13}\n")
        
        self.weights -= lr * weights_err
        self.bias -= lr * output_err
        return input_err

class ActivationLayer(Layer):
    def __init__(self, activation, d_activation):
        self.activation = activation
        self.d_activation = d_activation
    
    def forward_prop(self, input_data: npt.ArrayLike) -> npt.ArrayLike:
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output
    
    def backward_prop(self, output_err: npt.ArrayLike, lr) -> npt.ArrayLike:
        return self.d_activation(self.input) * output_err

### Network

In [136]:
from __future__ import annotations

class Network:
    
    def __init__(self):
        self.layers = []
        self.loss = None
        self.d_loss = None
    
    def add(self, layer: Layer):
        self.layers.append(layer)
    
    def use(self, loss, d_loss):
        self.loss = loss
        self.d_loss = d_loss
    
    def predict(self, input_data: npt.ArrayLike) -> list:
        samples = len(input_data)
        result = []
    
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_prop(output)
            result.append(output)
            
        return result
    
    def fit(self, x_train: npt.ArrayLike, y_train: npt.ArrayLike, epochs: int, lr: float):
        samples = len(x_train)
        
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_prop(output)
                
                err += self.loss(y_train[j], output)
                
                error = self.d_loss(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_prop(error, lr)
            
            err /= samples
            print(f"Epoch {i+1}/{epochs} >> error={err}")
        

### Util Functions

In [137]:
def ReLU(x):
    return np.clip(x, 0, None)

def d_ReLU(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def d_sigmoid(x):
    return (1 - sigmoid(x)) * sigmoid(x)

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def d_mse(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

### My Inputs

In [138]:
ACT_FUNCTIONS = {
    'ReLU': (ReLU, d_ReLU),
    'sigmoid': (sigmoid, d_sigmoid)
}

LEARNING_RATE = 0.1
EPOCHS = 1

### Network Inititation - Home Work I

In [182]:
def train_hw1_net(activation_function: str = 'ReLU') -> None:
    """
    Train the required net from HW1 Q2. During the training pipeline all required 
    outputs will be printed.
    """
    assert activation_function in ACT_FUNCTIONS.keys(), \
        "The provided activation function is available" 
        
    X_TRAIN = np.array([[[1, 2, -1]]])
    Y_TRAIN = np.array([[[0]]])
    ACT_FUNC = ACT_FUNCTIONS[activation_function]

    net_hw1 = Network()
    net_hw1.add(FullyConnectedLayer(3, 2, hw_1_init=True))
    net_hw1.add(ActivationLayer(*ACT_FUNC))
    net_hw1.add(FullyConnectedLayer(2, 2, hw_1_init=True))
    net_hw1.add(ActivationLayer(*ACT_FUNC))
    net_hw1.add(FullyConnectedLayer(2, 1, hw_1_init=True))

    net_hw1.use(mse, d_mse)
    net_hw1.fit(X_TRAIN, Y_TRAIN, epochs=EPOCHS, lr=LEARNING_RATE)
    
    ctr = 1
    for i, layer in enumerate(net_hw1.layers):
        if i in [0, 2, 4]:
            print(f"{'*'*13}")
            print(f'{" "*3}Layer {ctr}')
            print(f"{'*'*13}\n")
            print(f'Weights:\n{layer.weights}')
            print(f"{'-'*13}")
            print(f"Biases:\n{layer.bias}\n")
            ctr += 1

### ReLU

In [185]:
train_hw1_net(activation_function='ReLU')

*************
DLoss/DW:
[[64.]
 [64.]]
*************

*************
DLoss/DW:
[[32. 32.]
 [32. 32.]]
*************

*************
DLoss/DW:
[[ 32.  32.]
 [ 64.  64.]
 [-32. -32.]]
*************

Epoch 1/1 >> error=64.0
*************
   Layer 1
*************

Weights:
[[-2.2 -2.2]
 [-5.4 -5.4]
 [ 4.2  4.2]]
-------------
Biases:
[[-3.2 -3.2]]

*************
   Layer 2
*************

Weights:
[[-2.2 -2.2]
 [-2.2 -2.2]]
-------------
Biases:
[[-1.6 -1.6]]

*************
   Layer 3
*************

Weights:
[[-5.4]
 [-5.4]]
-------------
Biases:
[[-1.6]]



### Sigmoid

In [186]:
train_hw1_net(activation_function='sigmoid')

*************
DLoss/DW:
[[2.91322908]
 [2.91322908]]
*************

*************
DLoss/DW:
[[0.37614665 0.37614665]
 [0.37614665 0.37614665]]
*************

*************
DLoss/DW:
[[ 0.08967556  0.08967556]
 [ 0.17935112  0.17935112]
 [-0.08967556 -0.08967556]]
*************

Epoch 1/1 >> error=2.9132290817853628
*************
   Layer 1
*************

Weights:
[[0.99103244 0.99103244]
 [0.98206489 0.98206489]
 [1.00896756 1.00896756]]
-------------
Biases:
[[-0.00896756 -0.00896756]]

*************
   Layer 2
*************

Weights:
[[0.96238533 0.96238533]
 [0.96238533 0.96238533]]
-------------
Biases:
[[-0.04270526 -0.04270526]]

*************
   Layer 3
*************

Weights:
[[0.70867709]
 [0.70867709]]
-------------
Biases:
[[-0.34136368]]



# Question 4

    * Y =X1 −2X2 +3X3 −4X4 +ε

In [168]:
# Let's use gradient descent to learn the weights and bias that minimizes the loss function.
# For this, we need the gradient of the loss function and the gradients of the linear function.

class MSE:
    def __call__(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return ((y_pred - y_true) ** 2).mean()

    def backward(self):
        n = self.y_true.shape[0]
        self.gradient = 2. * (self.y_pred - self.y_true) / n
        return self.gradient


class Linear:
      def __init__(self, input_dim: int = 4, num_hidden: int = 1):
        self.weights = np.random.randn(input_dim, num_hidden)
        self.bias = np.zeros(num_hidden)
  
      def __call__(self, x):
        self.x = x
        output = x @ self.weights + self.bias
        return output

      def backward(self, gradient):
        self.weights_gradient = self.x.T @ gradient
        self.bias_gradient = gradient.sum(axis=0)
        self.x_gradient = gradient @ self.weights.T
        return self.x_gradient

      def update(self, lr: float = 0.1, decay_rate: float = None, itr: int = None):
            if decay_rate:
                lr = lr * np.power(decay_rate, itr)
            self.weights = self.weights - lr * self.weights_gradient
            self.bias = self.bias - lr * self.bias_gradient

In [165]:
DIM = (10000, 4)
MU, SIG = 0, 1
Y_COEF = [1, -2, 3, -4]

coef = np.array([Y_COEF])
X = np.random.uniform(0, 1, DIM)
eps = np.random.normal(MU, SIG, DIM[0])
y_true = np.array([(X @ coef.T)[:, 0] + eps]).T

In [292]:
def gradient_descent(X, y_true, conv_th: float = 0.001, lr: float = 0.01):
    diff = 100
    epoch = 0
    epoch_loss, epoch_loss_diff = {}, {} 
    
    loss = MSE()
    linear = Linear()
    
    while diff > conv_th:
        y_pred = linear(X)
        loss_value = loss(y_pred, y_true)
        epoch_loss.update({epoch: loss_value})
        diff = abs(epoch_loss[epoch] - epoch_loss[epoch-1]) \
            if len(epoch_loss) > 1 else abs(loss_value)
        epoch_loss_diff.update({epoch: diff})
        
        if epoch % 20 == 0:
            print(f'Epoch {epoch}, loss {loss_value}')

        gradient_from_loss = loss.backward()
        linear.backward(gradient_from_loss)
        linear.update(lr)
        epoch += 1
        

def gradient_descent(X,
                     y_true,
                     conv_th: float = 1e-5,
                     lr: float = 0.01,
                     gd_type: str = 'default',
                     decay_rate: float = None):
    diff = 100
    epoch = 0
    epoch_loss, epoch_loss_diff = {}, {} 
    
    loss = MSE()
    linear = Linear()
    
    while diff > conv_th:
        y_pred = linear(X)
        loss_value = loss(y_pred, y_true)
        epoch_loss.update({epoch: loss_value})
        diff = abs(epoch_loss[epoch] - epoch_loss[epoch-1]) \
            if len(epoch_loss) > 1 else abs(loss_value)
        epoch_loss_diff.update({epoch: diff})
        
        if epoch % 5 == 0:
            print(f'Epoch {epoch}, loss {loss_value},  diff {diff}')

        gradient_from_loss = loss.backward()
        linear.backward(gradient_from_loss)
        if 'exp' in gd_type:
            linear.update(lr, decay_rate=decay_rate if decay_rate else 0.8, itr=epoch)
        else:
            linear.update(lr)
        epoch += 1

In [339]:
def stochastic_gradient_descent(X,
                                y_true,
                                conv_th: float = 1e-5,
                                lr: float = 0.01,
                                batch_size: int = 1):
    batch_count = len(X)/batch_size
    diff = 100
    epoch = 0
    epoch_loss, epoch_loss_diff = {}, {} 
    
    loss = MSE()
    linear = Linear()
    
    while diff > conv_th and epoch < 100:
        data = list(zip(X, y_true))
        random.shuffle(data)
        X_tmp, Y_tmp = [np.asarray(item) for item in zip(*data)]
        for j in range(int(batch_count)):
            min_rng = int(j * batch_size)
            max_rng = int((j+1) * batch_size)

            X_batch = X_tmp[min_rng:max_rng]
            Y_batch = Y_tmp[min_rng:max_rng]
            
            y_pred = linear(X_batch)
            loss_value = loss(y_pred, Y_batch)
            gradient_from_loss = loss.backward()
            linear.backward(gradient_from_loss)
            linear.update(lr)
            
            print(f"Epoch {epoch}, Batch: {j}; Loss value: {loss_value}")
        
        if epoch % 5 == 0:
            print(f'Epoch {epoch}, loss {loss_value},  diff {diff}')
            
        epoch += 1
        epoch_loss.update({epoch: loss_value})
        diff = abs(epoch_loss[epoch] - epoch_loss[epoch-1]) \
            if len(epoch_loss) > 1 else abs(loss_value)
        epoch_loss_diff.update({epoch: diff})

In [341]:
# stochastic_gradient_descent(X, y_true, batch_size=1000)

In [170]:
loss = MSE()
linear = Linear()

num_epochs = 40
lr = 0.1

for epoch in range(num_epochs):
    y_pred = linear(X)
    loss_value = loss(y_pred, y_true)

    if epoch % 5 == 0:
        print(f'Epoch {epoch}, loss {loss_value}')
        gradient_from_loss = loss.backward()
        linear.backward(gradient_from_loss)
        linear.update(lr, decay_rate=0.95, itr=epoch)


Epoch 0, loss 2.8372332493347434
Epoch 5, loss 2.775910363370509
Epoch 10, loss 2.7301607676908333
Epoch 15, loss 2.6956708183166698
Epoch 20, loss 2.6695009000267995
Epoch 25, loss 2.6495520242170687
Epoch 30, loss 2.6342926182082738
Epoch 35, loss 2.62258948697412


In [155]:
import random
import numpy as np


def get_loss(X, Y, m, b):
    Y_pred = np.dot(m, X.T) + b
    RSS = ((Y - Y_pred.T) ** 2).sum()
    return RSS


def get_loss_grad(X, Y, m, b):
    Y_pred = np.dot(m, X.T) + b
    g_m = (-2) * (np.dot(X.T, (Y - Y_pred.T))).sum(axis=1)
    g_b = (-2) * (Y - Y_pred.T).sum()
    return g_m, g_b


def single_update(X, Y, m, b, learning_rate):
    g_m, g_b = get_loss_grad(X, Y, m, b)
    m -= learning_rate * g_m
    b -= learning_rate * g_b
    return m, b, g_m, g_b


def single_update_moment(X, Y, m, b, vm_old, vb_old, gamma, learning_rate):
    g_m, g_b = get_loss_grad(X, Y, m, b)
    v_m = gamma * vm_old + learning_rate * g_m
    v_b = gamma * vb_old + learning_rate * g_b
    m -= v_m
    b -= v_b
    return m, b, g_m, g_b, v_m, v_b


def GD(X, Y, m, b, learning_rate=1e-5, epochs=None, conv_thresh=1e-5):
    m_history = [m]
    b_history = [b]
    loss_history = [get_loss(X, Y, m, b)]
    count = 0
    diff = 100

    while (count < epochs) and (diff > conv_thresh):
        if count % 10 == 0:
            print(count)
        count += 1
        m, b, g_m, g_b = single_update(X, Y, m, b, learning_rate)
        m_history.append(m)
        b_history.append(b)
        curr_loss = get_loss(X, Y, m, b)
        diff = np.abs(curr_loss - loss_history[-1])
        print("Epoch {0}: loss: {1}, m: {2}, b: {3} ".format(str(count), str(curr_loss), str(m[0].tolist()), str(b)))
        loss_history.append(curr_loss)

    return m, b, m_history, b_history


def Exp_GD(X, Y, m, b, learning_rate=1e-5, decay_rate=0.8, epochs=None, conv_thresh=1e-5):
    m_history = [m]
    b_history = [b]
    loss_history = [get_loss(X, Y, m, b)]
    count = 0
    diff = 100
    original_learning_rate = learning_rate

    while (count < epochs) and (diff > conv_thresh):
        if count % 10 == 0:
            print(count)
        count += 1
        m, b, g_m, g_b = single_update(X, Y, m, b, learning_rate)
        m_history.append(m)
        b_history.append(b)
        curr_loss = get_loss(X, Y, m, b)
        diff = np.abs(curr_loss - loss_history[-1])
        print("Epoch {0}: loss: {1}, m: {2}, b: {3} ".format(str(count), str(curr_loss), str(m[0].tolist()), str(b)))
        loss_history.append(curr_loss)
        learning_rate = original_learning_rate * (np.power(decay_rate, count))
    return m, b, m_history, b_history


def SGD(X, Y, m, b, learning_rate=1e-5, batch_size=1, epochs=None, conv_thresh=1e-5):
    num_batches = len(X) / batch_size
    m_history = [m]
    b_history = [b]
    loss_history = [get_loss(X, Y, m, b)]
    count = 0
    diff = 100

    while (count < epochs) and (diff > conv_thresh):
        if count % 10 == 0:
            print(count)
        count += 1
        data = list(zip(X, Y))
        random.shuffle(data)
        X_temp, Y_temp = zip(*data)
        X_temp = np.asarray(X_temp)
        Y_temp = np.asarray(Y_temp)
        for j in range(int(num_batches) + 1):
            X_batch = X_temp[int(j * batch_size):int((j + 1) * batch_size)]
            Y_batch = Y_temp[int(j * batch_size):int((j + 1) * batch_size)]
            m, b, g_m, g_b = single_update(X_batch, Y_batch, m, b, learning_rate)
            curr_loss = get_loss(X, Y, m, b)
            m_history.append(m)
            b_history.append(b)
            # print( "Epoch {0} batch {1}: loss: {2}, m: {3}, b: {4} ".format(str(count),str(j), str(curr_loss),
            # str(m[0].tolist()), str(b)))
        curr_loss = get_loss(X, Y, m, b)
        diff = np.abs(curr_loss - loss_history[-1])
        print("Epoch {0}: loss: {1}, m: {2}, b: {3} ".format(str(count), str(curr_loss), str(m[0].tolist()), str(b)))
        loss_history.append(curr_loss)

    return m, b, m_history, b_history


def moment_GD(X, Y, m, b, gamma, learning_rate=1e-5, epochs=None, conv_thresh=1e-8):
    m_history = [m]
    b_history = [b]
    vm_history = [0]
    vb_history = [0]
    loss_history = [get_loss(X, Y, m, b)]
    count = 0
    diff = 100

    while (count < epochs) and (diff > conv_thresh):
        if count % 10 == 0:
            print(count)
        count += 1
        m, b, g_m, g_b, v_m, v_b = single_update_moment(X, Y, m, b, vm_history[-1], vb_history[-1], gamma,
                                                        learning_rate)
        m_history.append(m)
        b_history.append(b)
        vm_history.append(v_m)
        vb_history.append(v_b)
        curr_loss = get_loss(X, Y, m, b)
        diff = np.abs(curr_loss - loss_history[-1])
        print("Epoch {0}: loss: {1}, m: {2}, b: {3} ".format(str(count), str(curr_loss), str(m[0].tolist()), str(b)))
        loss_history.append(curr_loss)

    return m, b, m_history, b_history


if __name__ == '__main__':
    X = np.random.uniform(0, 1, (10000, 4))
    eps = np.random.normal(0, 1, 10000)
    Y = X[:, 0] - 2 * X[:, 1] + 3 * X[:, 2] - 4 * X[:, 3] + eps
    Y = np.reshape(Y, (Y.shape[0], 1))

    start_m = np.ones((1, 4))
    start_b = 1

    ##Regular Gradient Descent
    GD(X, Y, start_m, start_b, learning_rate=1e-5, epochs=2000)

    ##Exponential decay GD
    # Exp_GD(X, Y, start_m, start_b, learning_rate=1e-4, decay_rate=0.95, epochs=2000)

    ##SGD
    # SGD(X, Y, start_m, start_b, learning_rate=1e-4,batch_size=64, epochs=2000)

    ##Mpmentum GD
    # moment_GD(X, Y, start_m, start_b,gamma=0.8, learning_rate=1e-5,epochs=2000)


0
Epoch 1: loss: 91565.08132593942, m: [0.6004073283315883, 0.5474118097696812, 0.6329187620045054, 0.5214565915971918], b: 0.20058517831572364 
Epoch 2: loss: 52926.16757313269, m: [0.3726697308703131, 0.2679874724138139, 0.4362504729874872, 0.21451367227132684], b: -0.2692942782889725 
Epoch 3: loss: 38930.87802715722, m: [0.2464618150793518, 0.09121102482146437, 0.34007119668749863, 0.009492055656627613], b: -0.544202124927252 
Epoch 4: loss: 33579.63005179084, m: [0.18017268559615474, -0.02464699997269912, 0.3030105544149629, -0.13484582718734434], b: -0.7037638313674385 
Epoch 5: loss: 31269.72911210837, m: [0.14918267956837333, -0.10428213439253482, 0.30059412015588244, -0.24290763310015], b: -0.7950961356916345 
Epoch 6: loss: 30038.931684548937, m: [0.13892633847097174, -0.162310342087636, 0.31834524610023773, -0.3291421243159839], b: -0.8460793254769494 
Epoch 7: loss: 29199.701402362363, m: [0.14078754845641753, -0.20738313840796407, 0.3477031946946288, -0.4021049699719449], 